In [1]:
# reload code if library changes
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import numpy as np

In [3]:
import datajoint as dj
from workflow_calcium_imaging.pipeline import *
populate_settings = {'display_progress': True}

[2023-02-28 00:28:41,024][INFO]: Connecting jure@127.0.0.1:3306
[2023-02-28 00:28:41,125][INFO]: Connected jure@127.0.0.1:3306


In [4]:
# move to library
def get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init):
    all_subject_str = []
    all_session_str = [] # this will be list of lists - each nested list corresponding to one subject
    all_datetime_str = [] # for now hardcoded! (figure out how to do programmatically)

    count = 0
    for subject_str in os.listdir(root_data_dir):
        if os.path.isdir(f'{root_data_dir}/{subject_str}'):

            print(f'Subject: {subject_str}')
            all_subject_str.append(subject_str)

            all_subject_session_str = [] # sessions for this particular subject
            all_subject_datetime_str = []
            for subject_session_str in os.listdir(root_data_dir + '/' + subject_str):
                all_subject_session_str.append(subject_session_str)
                fake_session_datetime_str = fake_session_datetime_str_init[:18] + str(count) + '.000' # making fake unique time
                print('\n\n\nIMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.\n\n\n')
                print(fake_session_datetime_str)
                all_subject_datetime_str.append(fake_session_datetime_str) # here it is fake
                count += 1

            print(f'Identified sessions for subject {subject_str}: {all_subject_session_str}')

            all_session_str.append(all_subject_session_str) 
            all_datetime_str.append(all_subject_datetime_str) 
        
    return all_subject_str, all_session_str, all_datetime_str

### Clear previous s2p entries

In [5]:
imaging.ProcessingParamSet.delete()
imaging.Curation.delete()

In [6]:
# custom function to populate database
root_data_dir = dj.config['custom']['imaging_root_data_dir']
fake_session_datetime_str_init = '2002-01-01 12:00:00.000' # making up session datetime (to query data use either the session/folder name or PraireView metadata)
all_subject_str, all_session_str, all_datetime_str = get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init)


Subject: jm003



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:00.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:01.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:02.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the 

Automated Suite2p

In [7]:
choose_params = 'ops_jm_30hz_1plane' # number of channels, imaging rate and number of planes is overwritten by datajoint (see code for imaging.Processing.populate)

params_suite2p = np.load(f'/home/cossart/deve-networks/scripts/s2p_ops/{choose_params}.npy', allow_pickle=True).item()

params_suite2p['look_one_level_down'] = 0
params_suite2p['input_format'] = 'bruker'
params_suite2p['bruker'] = True
params_suite2p['force_sktiff'] = True
params_suite2p['two_step_registration'] = 0.0
params_suite2p['fast_disk'] = ['/suite2p_fast_disk']
params_suite2p

{'suite2p_version': '0.10.3',
 'look_one_level_down': 0,
 'fast_disk': [],
 'delete_bin': True,
 'mesoscan': False,
 'bruker': False,
 'bruker_bidirectional': False,
 'h5py': [],
 'h5py_key': 'data',
 'nwb_file': '',
 'nwb_driver': '',
 'nwb_series': '',
 'save_path0': [],
 'save_folder': [],
 'subfolders': [],
 'move_bin': False,
 'nplanes': 1,
 'nchannels': 1,
 'functional_chan': 1,
 'tau': 0.1,
 'fs': 30.0,
 'force_sktiff': True,
 'frames_include': -1,
 'multiplane_parallel': 0.0,
 'ignore_flyback': [],
 'preclassify': 0.0,
 'save_mat': False,
 'save_NWB': 0.0,
 'combined': 1.0,
 'aspect': 1.0,
 'do_bidiphase': False,
 'bidiphase': 0.0,
 'bidi_corrected': False,
 'do_registration': 1,
 'two_step_registration': 0.0,
 'keep_movie_raw': True,
 'nimg_init': 300,
 'batch_size': 500,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': True,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma_time': 0.0,
 'smooth_sigma': 1.15,
 'th_badframes': 1.0,
 'norm_frames': True,
 'force_refI

In [8]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='suite2p', 
    paramset_idx=0, 
    params=params_suite2p,
    paramset_desc=f'Calcium imaging analysis with Suite2p using {choose_params} parameters')

In [9]:
from workflow_calcium_imaging import process
process.run()


---- Populate imported and computed tables ----


Fluorescence:   0%|          | 0/1 [01:18<?, ?it/s]


LostConnectionError: Connection was lost during a transaction.

## Running suite2p for one animal

In [10]:
session.SessionDirectory()

subject,session_datetime,session_dir Path to the data directory for a session
jm003,2002-01-01 12:00:00,jm003/2022-05-03_b/TSeries-05032022-002
jm003,2002-01-01 12:00:01,jm003/2022-05-04_a/TSeries-05042022-001
jm003,2002-01-01 12:00:02,jm003/2022-05-04_b/TSeries-05042022-002
jm003,2002-01-01 12:00:03,jm003/2022-05-05_b/TSeries-05052022-002
jm003,2002-01-01 12:00:04,jm003/2022-05-05_c/TSeries-05052022-003
jm004,2002-01-01 12:00:05,jm004/2022-05-06_a/TSeries-05062022-001
jm007,2002-01-01 12:00:07,jm007/2022-06-13_b/TSeries-05182022-1006-002
jm007,2002-01-01 12:00:08,jm007/2022-06-14_a/TSeries-05182022-1006-001
jm007,2002-01-01 12:00:09,jm007/2022-06-14_b/TSeries-05182022-1006-002
jm007,2002-01-01 12:00:10,jm007/2022-06-15_a/TSeries-05182022-1006-002


In [11]:
session_key = (session.SessionDirectory & 'subject="jm007"').fetch('KEY')[0]
print(session_key )
session_dir_key = (session.SessionDirectory & 'subject="jm007"').fetch('session_dir')[0]

imaging.ProcessingTask.insert1(dict(session_key, 
                                    scan_id=0,
                                    paramset_idx=0,
                                    processing_output_dir=f'{session_dir_key}', 
                                    task_mode='trigger'))

{'subject': 'jm007', 'session_datetime': datetime.datetime(2002, 1, 1, 12, 0, 8)}


## Populate `imaging.Processing`

In [14]:
imaging.Processing.populate(**populate_settings);

In [15]:
imaging.Processing()

subject,session_datetime,scan_id,paramset_idx Uniqiue parameter set ID.,"processing_time Time of generation of this set of processed, segmented results",package_version
jm007,2002-01-01 12:00:07,0,0,2023-02-27 21:48:24,
jm007,2002-01-01 12:00:08,0,0,2023-02-28 04:00:58,


## Insert new Curation following the ProcessingTask

+ The next step in the pipeline is the curation of motion corection and segmentation results.

+ If a manual curation was implemented, an entry needs to be manually inserted into the table `imaging.Curation`, which specifies the directory to the curated results in `curation_output_dir`. 

+ If we would like to use the processed outcome directly, an entry is also needed in `imaging.Curation`. A method `create1_from_processing_task` was provided to help this insertion. It copies the `processing_output_dir` in `imaging.ProcessingTask` to the field `curation_output_dir` in the table `imaging.Curation` with a new `curation_id`.

    + In this example, we create/insert one `imaging.Curation` for each `imaging.ProcessingTask`, specifying the same output directory.

    + To this end, we could also make use of a convenient function `imaging.Curation().create1_from_processing_task()`

In [17]:
imaging.Curation.heading

# Curation(s) results
subject              : varchar(8)                   # 
session_datetime     : datetime                     # 
scan_id              : int                          # 
paramset_idx         : smallint                     # Uniqiue parameter set ID.
curation_id          : int                          # 
---
curation_time        : datetime                     # Time of generation of this set of curated results
curation_output_dir  : varchar(255)                 # Output directory of the curated results, relative to root data directory
manual_curation      : tinyint                      # Has manual curation been performed on this result?
curation_note=""     : varchar(2000)                # 

In [19]:
# temp just to try the 2 plane 2 channel:
import datetime
foo_dict = {'subject': 'jm007', 'session_datetime': datetime.datetime(2002, 1, 1, 12, 0, 8)}

imaging.Curation.insert1(dict(subject=foo_dict['subject'], 
                                session_datetime=foo_dict['session_datetime'], 
                                scan_id=0,
                                paramset_idx=0,
                                curation_id=0,
                                curation_time=foo_dict['session_datetime'], 
                                curation_output_dir='jm007/2022-06-14_a/TSeries-05182022-1006-001/suite2p',
                                manual_curation=False,
                                curation_note=''))

In [ ]:
for (i, subject_str) in enumerate(all_subject_str):
    #for (j, subject_session_str) in enumerate(all_session_str[i]): # ONLY RUNNING FOR FIRST SESSION
    
    j = 0
    subject_session_str = all_session_str[i][j]
    
    session_datetime_str = all_datetime_str[i][j]

    imaging.Curation.insert1(dict(subject=subject_str, 
                                    session_datetime=session_datetime_str, 
                                    scan_id=0,
                                    paramset_idx=0,
                                    curation_id=0,
                                    curation_time=session_datetime_str, 
                                    curation_output_dir=f'{subject_str}/{subject_session_str}/TSeries/suite2p',
                                    manual_curation=False,
                                    curation_note=''))


    

In [20]:
imaging.Activity()

subject,session_datetime,scan_id,paramset_idx Uniqiue parameter set ID.,curation_id,extraction_method


In [21]:
imaging.MotionCorrection.heading

# Results of motion correction
subject              : varchar(8)                   # 
session_datetime     : datetime                     # 
scan_id              : int                          # 
paramset_idx         : smallint                     # Uniqiue parameter set ID.
curation_id          : int                          # 
---
motion_correct_channel : tinyint                      # 0-based indexing

In [22]:
imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.MaskClassification.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)

MotionCorrection:   0%|          | 0/1 [00:00<?, ?it/s]

IMPORTANT: JM commented out the saving of non-rigid motion correction data.
 It seems like the new s2p version doesnt save s2p.ops["nblocks"][0] and similar fields (i documentation of the function supposed to do that (zalign) it says # This function doesnt work. Has a bunch of name errors
IMPORTANT: JM commented out the saving of non-rigid motion correction data.
 It seems like the new s2p version doesnt save s2p.ops["nblocks"][0] and similar fields (i documentation of the function supposed to do that (zalign) it says # This function doesnt work. Has a bunch of name errors
IMPORTANT: JM commented out the saving of non-rigid motion correction data.
 It seems like the new s2p version doesnt save s2p.ops["nblocks"][0] and similar fields (i documentation of the function supposed to do that (zalign) it says # This function doesnt work. Has a bunch of name errors


Fluorescence:   0%|          | 0/2 [01:13<?, ?it/s]


LostConnectionError: Connection was lost during a transaction.

# Testing if data is loaded correctly

In [ ]:
session_key = (imaging.Fluorescence & 'subject = "jm003"').fetch('KEY')[0]
print(session_key)

In [ ]:
imaging.MotionCorrection & session_key

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
avg_im

In [ ]:
avg_im = (imaging.MotionCorrection.Summary & session_key).fetch('average_image')
plt.imshow(np.array(avg_im[0]))
plt.show()
plt.imshow(np.array(avg_im[1]))
plt.show()
plt.imshow(np.array(avg_im[2]))
plt.show()

In [ ]:
from scipy.stats import zscore

In [ ]:
imaging.Activity.Trace & session_key

In [ ]:
F = (imaging.Fluorescence.Trace & session_key).fetch('fluorescence')

In [ ]:
F_toplot = zscore(np.stack(F), 1)

In [ ]:
plt.figure(figsize=(20,20), dpi=300)
plt.imshow(F_toplot,cmap='Greys', vmin=0, vmax=3.65)